In [566]:
import os
import sys
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd

In [567]:
df=pd.read_csv("car.csv")

In [568]:
###Convert this into Appropriate Format

In [569]:
c=0
global c
def func(x):
    global c
    c=c+1
    return c

In [570]:
df["ID"]=df["MilesDriven"].apply(func)

In [571]:
df["ID"]

0        1
1        2
2        3
3        4
4        5
      ... 
516    517
517    518
518    519
519    520
520    521
Name: ID, Length: 521, dtype: int64

In [572]:
df2=df.to_dict('records')

In [573]:
df2

[{'Brand': 'Nissan',
  'Model': 'Versa Note S Plus',
  'Year': 2017,
  'MilesDriven': '78,664',
  'Price': '$12,689 ',
  'EMI': '294',
  'City': 'Abington',
  'State': 'MA',
  'ID': 1},
 {'Brand': 'Buick',
  'Model': 'Envision Premium AWD',
  'Year': 2020,
  'MilesDriven': '16,107',
  'Price': '$32,488 ',
  'EMI': '719',
  'City': 'Medway',
  'State': 'MA',
  'ID': 2},
 {'Brand': 'Nissan',
  'Model': 'Rogue S AWD',
  'Year': 2020,
  'MilesDriven': '30,648',
  'Price': '$22,424 ',
  'EMI': '496',
  'City': 'Brockton',
  'State': 'MA',
  'ID': 3},
 {'Brand': 'Lincoln',
  'Model': 'Continental Reserve AWD',
  'Year': 2017,
  'MilesDriven': '41,252',
  'Price': '$28,381 ',
  'EMI': '657',
  'City': 'Abington',
  'State': 'MA',
  'ID': 4},
 {'Brand': 'Jeep',
  'Model': 'Grand Cherokee 4xe 4WD',
  'Year': 2022,
  'MilesDriven': '3,501',
  'Price': '$49,999 ',
  'EMI': '1,094',
  'City': 'Northborough',
  'State': 'MA',
  'ID': 5},
 {'Brand': 'Subaru',
  'Model': 'Crosstrek 2.0i Limited AWD',

In [574]:
len(df2)

521

In [575]:
def generator(df2):
    for c, line in enumerate(df2):
        yield {
    '_index':'carz',
    #'_type':'_doc',
    '_id':line.get('ID'),
    '_source': {
        'Brand':line.get('Brand',""),
        'Model':line.get('Model',""),
        'Year':line.get('Year',""),
        'MilesDriven':line.get('MilesDriven',""),
        'Price':line.get('Price',""),
        'EMI':line.get('EMI',""),
        'City':line.get('City',""),
        'State':line.get('State',"")
        
    }
            
        }
    #raise StopIteration

In [576]:
gen = generator(df2)

In [577]:
gen

<generator object generator at 0x0000029DBF950D50>

In [578]:
next(gen)

{'_index': 'carz',
 '_id': 1,
 '_source': {'Brand': 'Nissan',
  'Model': 'Versa Note S Plus',
  'Year': 2017,
  'MilesDriven': '78,664',
  'Price': '$12,689 ',
  'EMI': '294',
  'City': 'Abington',
  'State': 'MA'}}

In [579]:
es = Elasticsearch('http://localhost:9200')

In [580]:
settings={
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "Brand":{
                "type":"text"
            },
            "Model":{
                "type":"text"
            }
        }
    }
}

In [581]:
my=es.indices.create(index='carz')

In [582]:
my

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'carz'})

In [583]:
res=es.indices.get_alias()
for n in res:
    print(n)

.kibana_task_manager_8.11.1_001
.kibana_alerting_cases_8.11.1_001
.slo-observability.sli-v2
carz
.internal.alerts-ml.anomaly-detection.alerts-default-000001
.internal.alerts-observability.slo.alerts-default-000001
.kibana_ingest_8.11.1_001
.internal.alerts-observability.apm.alerts-default-000001
.internal.alerts-observability.metrics.alerts-default-000001
.slo-observability.summary-v2.temp
.apm-custom-link
.internal.alerts-security.alerts-default-000001
cargglllll
.transform-notifications-000002
carggllllll
.internal.alerts-stack.alerts-default-000001
.kibana_8.11.1_001
.internal.alerts-observability.logs.alerts-default-000001
.internal.alerts-observability.uptime.alerts-default-000001
.transform-internal-007
.apm-agent-configuration
.apm-source-map
.kibana_security_solution_8.11.1_001
.slo-observability.summary-v2
.kibana-observability-ai-assistant-conversations-000001
.internal.alerts-observability.threshold.alerts-default-000001
.kibana_analytics_8.11.1_001
.kibana-observability-ai-

C:\Users\sukum\AppData\Local\Temp\ipykernel_14932\1932514053.py:1: ElasticsearchWarning: this request accesses system indices: [.kibana_task_manager_8.11.1_001, .kibana_alerting_cases_8.11.1_001, .kibana_ingest_8.11.1_001, .apm-custom-link, .kibana_8.11.1_001, .transform-internal-007, .apm-agent-configuration, .kibana_security_solution_8.11.1_001, .kibana_analytics_8.11.1_001, .async-search], but in a future major version, direct access to system indices will be prevented by default
  res=es.indices.get_alias()


In [584]:
try:
    resp=helpers.bulk(es,generator(df2))
    print("working")
    print("Response:",resp)
except Exception as e:
    print(e)
    pass

working
Response: (521, [])
